# NMF Practical Work

**Umut Simsekli** 

Telecom Paris

## 1. Factorisation en matrices positives avec la $\beta$-divergence

$$
\begin{aligned}
\left ( {W*}, {H*} \right ) &= argmin_{W \leq 0, H \leq 0} \sum_{i=1}^{I} \sum_{j=1}^J d_{\beta}(x_{ij} || \hat{x}_{ij})
\\
\end{aligned}
$$
$$
\begin{aligned}
S &= \sum_{i=1}^{I} \sum_{j=1}^J d_{\beta}(x_{ij} || \hat{x}_{ij})
\\
&= \sum_{i=1}^{I} \sum_{j=1}^J d_{\beta}(x_{ij} || \hat{x}_{ij})
\\
\frac{\partial d_{\beta}(x_{ij} || \hat{x_{ij}})}{\partial w_{ik}} &= \hat{x}_{ij}^{\beta -1} - x_{ij}\hat{x_{ij}}^{\beta - 2}
\\
\frac{\partial d_{\beta}(x || \hat{x})}{\partial w_{ik}} &= \sum_{J=1}^{J} h_{kj} \left ( \hat{x}_{ij}^{\beta -1} - x_{ij}\hat{x_{ij}}^{\beta  - 2} \right )
\\
&= \left ( \sum_{j=1}^{J} h_{kj} \hat{x}_{ij}^{\beta - 1} \right ) - \left ( \sum_{j=1}^{J} h_{kj} x_{ij} \hat{x}{ij}^{\beta - 2} \right )
&= p_{ik} - m_{ik}
\end{aligned}
$$
Avec $p_{ik}, m_{ik} \geq 0$, on sait mettre à jour $W$ et $H$ de la sorte :
$$
\begin{aligned}
w_{ik} & \leftarrow w_{ik} \frac{m_{ik}}{p_{ik}}
\\
w_{ik} & \leftarrow w_{ik} \frac{\sum_{j=1}^{J} x_{ij}\hat{x_{ij} }^{\beta - 2} h_{kj}}{\sum_{j=1}^{J} \hat{x}_{ij}^{\beta -1} h_{kj}}
\\
h_{kj} & \leftarrow h_{kj} \frac{\sum_{i=1}^{I} x_{ij}\hat{x_{ij} }^{\beta - 2} w_{ik}}{\sum_{i=1}^{I} \hat{x}_{ij}^{\beta -1} w_{ik}}
\end{aligned}
$$

On doit choisir la fenêtre de Hann (largueur $4/M$ dans Fourier)
avec une longueur temporelle $M$ telle que l'on a $M=80 / f_e$.

## 2. Variantes autour d'un exemple simple

### 2. 1 Construction de l'exemple simple

In [ ]:
import numpy as np
import numpy.typing as npt

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

# and IPython.display for audio output
import IPython.display as ipd

# Librosa for audio
import librosa
# # And the display module for visualization
import librosa.display


In [ ]:
# Audio file
fname = 'Cmajor_piano8khz.wav'

# Load signal
y, sr = librosa.load(fname)

# Compute STFT
Xstft = librosa.stft(y, n_fft=2048, hop_length=512, win_length=None)
X = np.abs(Xstft)

#Visualize the waveform
plt.figure()
plt.figure(figsize=(14, 5))
librosa.display.waveplot(y, sr=sr)

#Visualize the spectrum
plt.figure()
plt.imshow(X**0.3)
plt.gca().invert_yaxis()


In [ ]:
# Implement MUR for NMF with beta-divergence 



In [ ]:
# Use Wiener filtering to reconstruct each note separately


# Example reconstruction
y_hat = librosa.istft(Xstft)
ipd.Audio(y_hat, rate=sr)
